In [1]:
import pandas as pd
import numpy as np

# Feature Preparation
This script starts with the filtered data and ends with the aggregated group features

In [ ]:
# read data
feature_data = pd.read_excel('unprepared_data.xlsx', sheet_name='Sheet1')
label_data = pd.read_excel('Clean_Keys.xlsx')

In [3]:
# merge features abd labels
merged_data = feature_data.merge(
    label_data,
    how='left',
    left_on='Sender',
    right_on='Anonymized Name'
)
merged_data = merged_data.dropna(subset=["Trabajo Final"])
merged_data.reset_index(inplace=True)

In [4]:
dta_dummies = pd.get_dummies(merged_data["Message Group DTA"], prefix="DTA")
merged_data = pd.concat([merged_data, dta_dummies], axis=1)

# Grouping and Aggregating
aggregated_data = merged_data.groupby(["Team", "Trabajo Final"]).agg({
    # Numeric Features
    "Sentiment Stars": ["mean", "std", "max", "min", "median", 
                        lambda x: np.percentile(x, 75) - np.percentile(x, 25), 
                        lambda x: x.skew(), 
                        lambda x: x.kurtosis(), 
                        lambda x: x.max() - x.min()],
    
    "Berger Score": ["mean", "std", "max", "min", "median", 
                     lambda x: np.percentile(x, 75) - np.percentile(x, 25), 
                     lambda x: x.skew(), 
                     lambda x: x.kurtosis(), 
                     lambda x: x.max() - x.min()],
    
    "Semantic Similarity": ["mean", "std", "max", "min", "median", 
                            lambda x: np.nanpercentile(x, 75) - np.nanpercentile(x, 25), 
                            lambda x: x.skew(), 
                            lambda x: x.dropna().kurtosis() if not x.dropna().empty else 0, 
                            lambda x: x.max() - x.min()],
    
    "Word_Count": ["mean", "std", "max", "min", "median", 
                   lambda x: np.percentile(x, 75) - np.percentile(x, 25), 
                   lambda x: x.skew(), 
                   lambda x: x.kurtosis(), 
                   lambda x: x.max() - x.min()],

    "Gini_Coefficient_Team": ["mean", "std", "max", "min", "median", 
                   lambda x: np.percentile(x, 75) - np.percentile(x, 25), 
                   lambda x: x.skew(), 
                   lambda x: x.kurtosis(), 
                   lambda x: x.max() - x.min()],

    "Openness": ["mean", "std", "max", "min", "median", 
                   lambda x: np.percentile(x, 75) - np.percentile(x, 25), 
                   lambda x: x.skew(), 
                   lambda x: x.kurtosis(), 
                   lambda x: x.max() - x.min()],

    "Conscientiousness": ["mean", "std", "max", "min", "median", 
                   lambda x: np.percentile(x, 75) - np.percentile(x, 25), 
                   lambda x: x.skew(), 
                   lambda x: x.kurtosis(), 
                   lambda x: x.max() - x.min()],

    "Extraversion": ["mean", "std", "max", "min", "median", 
                   lambda x: np.percentile(x, 75) - np.percentile(x, 25), 
                   lambda x: x.skew(), 
                   lambda x: x.kurtosis(), 
                   lambda x: x.max() - x.min()],

    "Agreeableness": ["mean", "std", "max", "min", "median", 
                   lambda x: np.percentile(x, 75) - np.percentile(x, 25), 
                   lambda x: x.skew(), 
                   lambda x: x.kurtosis(), 
                   lambda x: x.max() - x.min()],

    "Neuroticism": ["mean", "std", "max", "min", "median", 
                   lambda x: np.percentile(x, 75) - np.percentile(x, 25), 
                   lambda x: x.skew(), 
                   lambda x: x.kurtosis(), 
                   lambda x: x.max() - x.min()],

    
    # Other Binary Aggregations
    "ContainsInappropriateWords": ["count", "sum", "mean"],
    "IsWeekend": ["sum", "mean"],
})

# Flatten MultiIndex column names
aggregated_data.columns = ["_".join(col).strip() for col in aggregated_data.columns]

# Aggregating DTA categories
dta_aggregation = merged_data.groupby(["Team", "Trabajo Final"])[dta_dummies.columns].agg(["sum", "mean"])

# Flatten column names for DTA aggregation
dta_aggregation.columns = ["_".join(col).strip() for col in dta_aggregation.columns]

# Merge DTA aggregation with the main aggregated data
aggregated_data = aggregated_data.merge(dta_aggregation, on=["Team", "Trabajo Final"], how="left")

aggregated_data.fillna(0, inplace=True)

# Reset index
aggregated_data = aggregated_data.reset_index()

# Custom renaming for lambda-generated columns
rename_dict = {
    "Sentiment Stars_<lambda_0>": "Sentiment Stars_IQR",
    "Sentiment Stars_<lambda_1>": "Sentiment Stars_skew",
    "Sentiment Stars_<lambda_2>": "Sentiment Stars_kurtosis",
    "Sentiment Stars_<lambda_3>": "Sentiment Stars_range",
    
    "Berger Score_<lambda_0>": "Berger Score_IQR",
    "Berger Score_<lambda_1>": "Berger Score_skew",
    "Berger Score_<lambda_2>": "Berger Score_kurtosis",
    "Berger Score_<lambda_3>": "Berger Score_range",
    
    "Semantic Similarity_<lambda_0>": "Semantic Similarity_IQR",
    "Semantic Similarity_<lambda_1>": "Semantic Similarity_skew",
    "Semantic Similarity_<lambda_2>": "Semantic Similarity_kurtosis",
    "Semantic Similarity_<lambda_3>": "Semantic Similarity_range",
    
    "Word_Count_<lambda_0>": "Word_Count_IQR",
    "Word_Count_<lambda_1>": "Word_Count_skew",
    "Word_Count_<lambda_2>": "Word_Count_kurtosis",
    "Word_Count_<lambda_3>": "Word_Count_range",

    "Gini_Coefficient_Team_<lambda_0>": "Gini_Coefficient_Team_IQR",
    "Gini_Coefficient_Team_<lambda_1>": "Gini_Coefficient_Team_skew",
    "Gini_Coefficient_Team_<lambda_2>": "Gini_Coefficient_Team_kurtosis",
    "Gini_Coefficient_Team_<lambda_3>": "Gini_Coefficient_Team_range",

    "Openness_<lambda_0>": "Openness_IQR",
    "Openness_<lambda_1>": "Openness_skew",
    "Openness_<lambda_2>": "Openness_kurtosis",
    "Openness_<lambda_3>": "Openness_range",

    "Conscientiousness_<lambda_0>": "Conscientiousness_IQR",
    "Conscientiousness_<lambda_1>": "Conscientiousness_skew",
    "Conscientiousness_<lambda_2>": "Conscientiousness_kurtosis",
    "Conscientiousness_<lambda_3>": "Conscientiousness_range",

    "Extraversion_<lambda_0>": "Extraversion_IQR",
    "Extraversion_<lambda_1>": "Extraversion_skew",
    "Extraversion_<lambda_2>": "Extraversion_kurtosis",
    "Extraversion_<lambda_3>": "Extraversion_range",

    "Agreeableness_<lambda_0>": "Agreeableness_IQR",
    "Agreeableness_<lambda_1>": "Agreeableness_skew",
    "Agreeableness_<lambda_2>": "Agreeableness_kurtosis",
    "Agreeableness_<lambda_3>": "Agreeableness_range",

    "Neuroticism_<lambda_0>": "Neuroticism_IQR",
    "Neuroticism_<lambda_1>": "Neuroticism_skew",
    "Neuroticism_<lambda_2>": "Neuroticism_kurtosis",
    "Neuroticism_<lambda_3>": "Neuroticism_range",

    "ContainsInappropriateWords_count": "Message_Count"
}

# Rename the columns
aggregated_data.rename(columns=rename_dict, inplace=True)

In [5]:
# save file
file_name = "aggregated_data.xlsx"
aggregated_data.to_excel(file_name, index=False)